<a href="https://colab.research.google.com/github/mohamedamr13/Machine-Learning-/blob/main/NLP_session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras
import numpy as np

**Text Understanding**

In [ ]:
# Retrieve the training sequences.
(x_train, y_train), (x_test,y_test) = tf.keras.datasets.imdb.load_data()
# Retrieve the word index file mapping words to indices
word_index = keras.datasets.imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
inverted_word_index = dict((i, word) for (word, i) in word_index.items())
# Decode the first sequence in the dataset
decoded_sequence = " ".join(inverted_word_index[i] for i in x_train[0])

In [ ]:
x_train.shape

(25000,)

In [ ]:
vocab_size = len(word_index)


In [ ]:
bag_of_words_x_train = []
for sentence in x_train[:2000]:
    temp = [0 for _ in range(vocab_size+1)]
    for word in sentence:
        temp[word]+=1
    bag_of_words_x_train.append(temp)
bag_of_words_x_train = np.array(bag_of_words_x_train)

In [ ]:
bag_of_words_x_test = []
for sentence in x_test[:100]:
    temp = [0 for _ in range(vocab_size+1)]
    for word in sentence:
        temp[word]+=1
    bag_of_words_x_test.append(temp)
bag_of_words_x_test = np.array(bag_of_words_x_test)

In [ ]:
bag_of_words_x_train.shape, bag_of_words_x_test.shape

((2000, 88585), (100, 88585))

In [ ]:
max_seq = 0
min_seq = 1000000
for sentence in x_train[:2000]:
    max_seq = max(max_seq, len(sentence))
    min_seq = min(min_seq, len(sentence))
print(max_seq, min_seq)

1038 19


In [ ]:
pad_character = vocab_size + 1
padded_x_train = []
for sentence in x_train[:2000]:
    while(len(sentence) < max_seq):
        sentence.append(pad_character)
    padded_x_train.append(np.array(sentence))
padded_x_train = np.array(padded_x_train)
padded_y_train = y_train[:2000]

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Dense(1024,input_shape=(vocab_size+1,),activation='relu'))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(256,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid')) ## why not tanh




In [ ]:
model.fit(bag_of_words_x_train,y_train[:2000], validation_data = (bag_of_words_x_test,y_test[:2000]), epochs=10, batch_size = 16)

Epoch 1/10
125/125 [==============================] - ETA: 0s - loss: 0.0407 - accuracy: 0.9890

In [ ]:
# model_lstm = keras.models.Sequential()
input_layer = keras.layers.Input((max_seq,))
embedding = keras.layers.Embedding(vocab_size+1, 64,)(input_layer)

lstm_layer = keras.layers.LSTM(16, return_sequences= True)(embedding)
lstm_layer_2 = keras.layers.LSTM(16, return_sequences= False)(lstm_layer)

output_layer = keras.layers.Dense(1,activation='sigmoid')(lstm_layer_2)
model_lstm = keras.models.Model(input_layer,output_layer)

model_lstm.compile(loss="binary_crossentropy",optimizer='adam',metrics=["accuracy"])


In [ ]:
model_lstm.fit(padded_x_train,padded_y_train, epochs=10, batch_size = 16)

Epoch 1/10
125/125 [==============================] - 9s 46ms/step - loss: 0.6936 - accuracy: 0.5075
Epoch 2/10
125/125 [==============================] - 6s 48ms/step - loss: 0.6934 - accuracy: 0.4985
Epoch 3/10
125/125 [==============================] - 6s 44ms/step - loss: 0.6931 - accuracy: 0.5125
Epoch 4/10
125/125 [==============================] - 6s 45ms/step - loss: 0.6930 - accuracy: 0.5120
Epoch 5/10
125/125 [==============================] - 6s 45ms/step - loss: 0.6932 - accuracy: 0.5130
Epoch 6/10
125/125 [==============================] - 6s 44ms/step - loss: 0.6939 - accuracy: 0.5000
Epoch 7/10
125/125 [==============================] - 6s 45ms/step - loss: 0.6924 - accuracy: 0.5150
Epoch 8/10
125/125 [==============================] - 6s 46ms/step - loss: 0.6968 - accuracy: 0.5120
Epoch 9/10
125/125 [==============================] - 6s 45ms/step - loss: 0.6938 - accuracy: 0.5010
Epoch 10/10
125/125 [==============================] - 6s 44ms/step - loss: 0.6931 - accura

**Text Generation**

In [ ]:
pred = model_lstm.predict(padded_x_train[:1])

In [ ]:
pred.shape

(1, 1038, 16)

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

Stripping

In [ ]:
text = text.replace("\n",' ').replace(':',"").replace(";","").replace(".","").replace(",","") #.......
text = text.lower()


Tokenization

In [ ]:
word_tokenized_text = text.split(" ")
print(word_tokenized_text)

['first', 'citizen', 'before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', 'all', 'speak', 'speak', '', 'first', 'citizen', 'you', 'are', 'all', 'resolved', 'rather', 'to', 'die', 'than', 'to', 'famish?', '', 'all', 'resolved', 'resolved', '', 'first', 'citizen', 'first', 'you', 'know', 'caius', 'marcius', 'is', 'chief', 'enemy', 'to', 'the', 'people', '', 'all', 'we', "know't", 'we', "know't", '', 'first', 'citizen', 'let', 'us', 'kill', 'him', 'and', "we'll", 'have', 'corn', 'at', 'our', 'own', 'price', "is't", 'a', 'verdict?', '', 'all', 'no', 'more', 'talking', "on't", 'let', 'it', 'be', 'done', 'away', 'away!', '', 'second', 'citizen', 'one', 'word', 'good', 'citizens', '', 'first', 'citizen', 'we', 'are', 'accounted', 'poor', 'citizens', 'the', 'patricians', 'good', 'what', 'authority', 'surfeits', 'on', 'would', 'relieve', 'us', 'if', 'they', 'would', 'yield', 'us', 'but', 'the', 'superfluity', 'while', 'it', 'were', 'wholesome', 'we', 'might', 'guess', 'they',

In [ ]:
char_tokenized_text = []
for char in text[:100_000]:
  if(len(char)<1):
    continue
  char_tokenized_text.append(char)
print(char_tokenized_text)

['f', 'i', 'r', 's', 't', ' ', 'c', 'i', 't', 'i', 'z', 'e', 'n', ' ', 'b', 'e', 'f', 'o', 'r', 'e', ' ', 'w', 'e', ' ', 'p', 'r', 'o', 'c', 'e', 'e', 'd', ' ', 'a', 'n', 'y', ' ', 'f', 'u', 'r', 't', 'h', 'e', 'r', ' ', 'h', 'e', 'a', 'r', ' ', 'm', 'e', ' ', 's', 'p', 'e', 'a', 'k', ' ', ' ', 'a', 'l', 'l', ' ', 's', 'p', 'e', 'a', 'k', ' ', 's', 'p', 'e', 'a', 'k', ' ', ' ', 'f', 'i', 'r', 's', 't', ' ', 'c', 'i', 't', 'i', 'z', 'e', 'n', ' ', 'y', 'o', 'u', ' ', 'a', 'r', 'e', ' ', 'a', 'l', 'l', ' ', 'r', 'e', 's', 'o', 'l', 'v', 'e', 'd', ' ', 'r', 'a', 't', 'h', 'e', 'r', ' ', 't', 'o', ' ', 'd', 'i', 'e', ' ', 't', 'h', 'a', 'n', ' ', 't', 'o', ' ', 'f', 'a', 'm', 'i', 's', 'h', '?', ' ', ' ', 'a', 'l', 'l', ' ', 'r', 'e', 's', 'o', 'l', 'v', 'e', 'd', ' ', 'r', 'e', 's', 'o', 'l', 'v', 'e', 'd', ' ', ' ', 'f', 'i', 'r', 's', 't', ' ', 'c', 'i', 't', 'i', 'z', 'e', 'n', ' ', 'f', 'i', 'r', 's', 't', ' ', 'y', 'o', 'u', ' ', 'k', 'n', 'o', 'w', ' ', 'c', 'a', 'i', 'u', 's', ' ',

In [ ]:
vectorized_char_text = []
char2idx = dict()
idx2char = dict()
i = 0
for char in char_tokenized_text: 
  if(char not in char2idx):
      char2idx[char] = i
      idx2char[i] = char
      i+=1
char2idx

{' ': 5,
 '!': 26,
 '&': 31,
 "'": 24,
 '-': 28,
 '?': 23,
 'a': 15,
 'b': 10,
 'c': 6,
 'd': 14,
 'e': 8,
 'f': 0,
 'g': 25,
 'h': 18,
 'i': 1,
 'j': 27,
 'k': 20,
 'l': 21,
 'm': 19,
 'n': 9,
 'o': 11,
 'p': 13,
 'q': 29,
 'r': 2,
 's': 3,
 't': 4,
 'u': 17,
 'v': 22,
 'w': 12,
 'x': 30,
 'y': 16,
 'z': 7}

In [ ]:
vectorized_word_text = []
word2idx = dict()
idx2word = dict()
i = 0
for word in word_tokenized_text: 
  if(word not in word2idx):
      word2idx[word] = i
      idx2word[i] = word
      i+=1
word2idx

In [ ]:
word_x_train = []
word_y_train = []

prev_words = 5
for i in range(0,len(word_tokenized_text)-6,5):
    x = word_tokenized_text[i:i+5]
    y = word_tokenized_text[i+5]
    word_x_train.append([word2idx[w] for w in x])
    word_y_train.append(word2idx[y])


In [ ]:
word_x_train = np.array(word_x_train)
word_y_train = np.array(word_y_train)
print(word_x_train.shape, word_y_train.shape)

(41978, 5) (41978,)


In [ ]:
char_x_train = []
char_y_train = []

prev_chars = 10
for i in range(0,len(char_tokenized_text)-(prev_chars+1),prev_chars):
    x = char_tokenized_text[i:i+prev_chars]
    y = char_tokenized_text[i+prev_chars]
    char_x_train.append([char2idx[c] for c in x])
    char_y_train.append(char2idx[y])
char_x_train = np.array(char_x_train)
char_y_train = np.array(char_y_train)

In [ ]:
print(char_x_train.shape, char_y_train.shape)

(9999, 10) (9999,)


In [ ]:
vocab_size = len(word2idx)
vocab_size

15456

In [ ]:
# model_lstm = keras.models.Sequential()
input_layer = keras.layers.Input((prev_words,))
embedding = keras.layers.Embedding(in_vocab_size+1, 64,)(input_layer)

lstm_layer = keras.layers.LSTM(16, return_sequences= True)(embedding)
lstm_layer_2 = keras.layers.LSTM(out_vocab_size, return_sequences= False)(lstm_layer)

# output_layer = keras.layers.Dense(vocab_size,activation='softmax')(lstm_layer_2)
model_lstm = keras.models.Model(input_layer,output_layer)

model_lstm.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=["accuracy"])


In [ ]:
pred = model_lstm.predict(word_x_train[:1])
idx2word[np.argmax(pred)]


'spectatorship'

In [ ]:
def get_sentence(vectorized_words):
  sentence = ""
  for vector in vectorized_words:
    sentence += idx2word[vector] + ' '
  return sentence

In [ ]:
get_sentence(word_x_train[0])

'first citizen before we proceed '

In [ ]:
model_lstm.fit(word_x_train,word_y_train, epochs=10, batch_size = 16)

Epoch 1/10
2624/2624 [==============================] - 19s 6ms/step - loss: 7.1477 - accuracy: 0.0335
Epoch 2/10
2624/2624 [==============================] - 16s 6ms/step - loss: 6.6017 - accuracy: 0.0355
Epoch 3/10
2624/2624 [==============================] - 17s 6ms/step - loss: 6.4277 - accuracy: 0.0434
Epoch 4/10
2624/2624 [==============================] - 16s 6ms/step - loss: 6.2591 - accuracy: 0.0485
Epoch 5/10
2624/2624 [==============================] - 16s 6ms/step - loss: 6.1058 - accuracy: 0.0545
Epoch 6/10
2624/2624 [==============================] - 16s 6ms/step - loss: 5.9526 - accuracy: 0.0613
Epoch 7/10
2624/2624 [==============================] - 17s 6ms/step - loss: 5.8079 - accuracy: 0.0684
Epoch 8/10
2624/2624 [==============================] - 16s 6ms/step - loss: 5.6671 - accuracy: 0.0778
Epoch 9/10
2624/2624 [==============================] - 16s 6ms/step - loss: 5.5312 - accuracy: 0.0875
Epoch 10/10
2624/2624 [==============================] - 16s 6ms/step - l

In [ ]:
vocab_size = len(char2idx)
vocab_size

32

In [ ]:
# model_lstm = keras.models.Sequential()
input_layer = keras.layers.Input((prev_chars,))
embedding = keras.layers.Embedding(vocab_size+1, 64,)(input_layer)

lstm_layer = keras.layers.LSTM(16, return_sequences= True)(embedding)
lstm_layer_2 = keras.layers.LSTM(16, return_sequences= False)(lstm_layer)

output_layer = keras.layers.Dense(vocab_size,activation='softmax')(lstm_layer_2)
model_lstm = keras.models.Model(input_layer,output_layer)

model_lstm.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=["accuracy"])


In [ ]:
model_lstm.fit(char_x_train,char_y_train, epochs=10, batch_size = 16)

Epoch 1/10
625/625 [==============================] - 6s 6ms/step - loss: 2.9086 - accuracy: 0.1828
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 2.7477 - accuracy: 0.2110
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 2.5982 - accuracy: 0.2524
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 2.4925 - accuracy: 0.2860
Epoch 5/10
625/625 [==============================] - 3s 6ms/step - loss: 2.4322 - accuracy: 0.3050
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 2.3912 - accuracy: 0.3130
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 2.3564 - accuracy: 0.3204
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 2.3234 - accuracy: 0.3270
Epoch 9/10
625/625 [==============================] - 3s 6ms/step - loss: 2.2917 - accuracy: 0.3403
Epoch 10/10
625/625 [==============================] - 3s 5ms/step - loss: 2.2613 - accuracy: 0.3494